# Testing and Troubleshooting TCLab

To test and troubleshoot problems with TCLab, download this notebook to your laptop and run the following cells. 

## Testing

The next cells test for some commonly encountered issues with installation and operation of the Temperature Control Laboratory. This approach uses the library [`ipytest`](https://github.com/chmp/ipytest) which adapts the well known `pytest` unit testing environment for use within Jupyter notebooks. If you have not encountered unit testing before, it is a widely used technique to check for correct operation of individual units of code.

Run these cells. If the any of unit tests fail, comments within the unit test code may help determine the reason for the failure.

### Installing ipytest

The following cell will install, if needed, a copy of the `ipytest` library. This cell will produce no output.

In [1]:
try:
    import ipytest
except:
    !pip install ipytest
    import ipytest
    
ipytest.autoconfig()

### Software testing

Run the following tests to check for correct installation of the `tclab` library. These tests do not require the `tclab` hardware to be connected. If either of these tests fail, try reinstalling the `tclab` library.

In [2]:
%%ipytest --verbosity=1

# Verify tclab has been installed and is accessible by the Python kernal.
    
def test_tclab_install():
    from tclab import TCLab, clock, Historian, Plotter
    
# Verify that TCLab can be run offline (i.e., without access to hardware).

def test_tclab_offline():
    from tclab import setup
    TCLab = setup(connected=False, speedup=20)
    with TCLab() as lab:
        pass

======================================= test session starts ========================================
platform darwin -- Python 3.9.15, pytest-7.2.1, pluggy-1.0.0 -- /Users/jeff/opt/anaconda3/bin/python
cachedir: .pytest_cache
rootdir: /Users/jeff/Google Drive/GitHub/cbe30338-book/tclab
plugins: anyio-3.6.2
collecting ... collected 2 items

t_631f173c8a5d4cbc9c37369d2ded87c3.py::test_tclab_install PASSED                             [ 50%]
t_631f173c8a5d4cbc9c37369d2ded87c3.py::test_tclab_offline PASSED                             [100%]

======================================== 2 passed in 0.03s =========================================


### Hardware testing

The following tests require the `tclab` hardware to be connected to your laptop. The tests check for connectivity and operation of the hardware. The Temperature Control Lab must be connected to pass these tests. The tests check for

1. Verify we are not trying to attach hardware to Google Colab. 
2. Verify we can open a connection to the Arduino. If this fails, try detaching the Arduino, then restarting the Python kernel, and reattaching the Arduino.
3. Test to be sure we are running a recent version of the TCLab firmware on the Arduino. If this fails you need to update the Arduino firmware.




In [3]:
%%ipytest --verbosity=1

# TCLab cannot access the Arduino if it is run remotely on Google Colab. This is a common
# error since notebooks are so easy to open in Google Colab. The following test fails if
# the test is run on Google Colab.

def test_not_google_colab():
    import sys
    assert not "google.colab" in sys.modules

# Verify that a connection an be opened to the Arduino.

def test_tclab_connect():
    from tclab import TCLab, clock, Historian, Plotter
    lab = TCLab()
    lab.close()
    
# Verify tclab-sketch firmware version number is 2.0.1 or greater

def test_tclab_firmware_version():
    import packaging
    import re
    from tclab import TCLab
    with TCLab() as lab:
        vers = re.search(r'\s*([\d.]+)', lab.version).group(1)
    assert packaging.version.parse(vers) >= packaging.version.parse("2.0.1")

======================================= test session starts ========================================
platform darwin -- Python 3.9.15, pytest-7.2.1, pluggy-1.0.0 -- /Users/jeff/opt/anaconda3/bin/python
cachedir: .pytest_cache
rootdir: /Users/jeff/Google Drive/GitHub/cbe30338-book/tclab
plugins: anyio-3.6.2
collecting ... collected 3 items

t_631f173c8a5d4cbc9c37369d2ded87c3.py::test_not_google_colab PASSED                          [ 33%]
t_631f173c8a5d4cbc9c37369d2ded87c3.py::test_tclab_connect PASSED                             [ 66%]
t_631f173c8a5d4cbc9c37369d2ded87c3.py::test_tclab_firmware_version PASSED                    [100%]

======================================== 3 passed in 8.88s =========================================


### Visual test

If all of the above tests pass, then the following code fragment should turn on the LED at 50% level for 10 seconds.

In [4]:
from tclab import TCLab

lab = TCLab()
lab.LED(50)
lab.close()

TCLab version 0.4.10dev
Arduino Leonardo connected on port /dev/cu.usbmodem144101 at 115200 baud.
TCLab Firmware 2.0.1 Arduino Leonardo/Micro.
TCLab disconnected successfully.


## Running Diagnostics

In [5]:
import tclab

print("Version = ", tclab.__version__)
tclab.diagnose()

Version =  0.4.10dev

Checking connection
-------------------
Looking for Arduino on any port...
Arduino Leonardo found on port /dev/cu.usbmodem144101

Testing TCLab object in debug mode
----------------------------------
TCLab version 0.4.10dev
Sent: "Q1 0"
Return: "0.000"
Sent: "VER"
Return: "TCLab Firmware 2.0.1 Arduino Leonardo/Micro"
Arduino Leonardo connected on port /dev/cu.usbmodem144101 at 115200 baud.
TCLab Firmware 2.0.1 Arduino Leonardo/Micro.
Sent: "Q2 0"
Return: "0.000"
Reading temperature
Sent: "T1"
Return: "23.154"
23.154
Sent: "Q1 0"
Return: "0.000"
Sent: "Q2 0"
Return: "0.000"
Sent: "X"
Return: "Stop"
TCLab disconnected successfully.

Testing TCLab functions
-----------------------
TCLab version 0.4.10dev
Arduino Leonardo connected on port /dev/cu.usbmodem144101 at 115200 baud.
TCLab Firmware 2.0.1 Arduino Leonardo/Micro.
Testing LED. Should turn on for 10 seconds.
Countdown: 0  

Reading temperatures
T1 = 23.154 °C, T2 = 22.188 °C

Writing fractional value to heaters

## Troubleshooting

### Laptop doesn't connect to the Arduino

If your laptop consistently fails to locate or connect to the Arduino device, check the following:

1. Be sure you are have installed Python and tclab on your laptop, and that you are not attempting to run tclab on a remote server such as Google Colab. tclab requires direct access to the USB port on your laptop.

1. Check cable connections. The USB cable from the Arduino should be connected to your laptop. The USB cable for the heater shield should be connected to a USB power supply. Reversing these connections will not harm anything but the system will not function.

1. Verify that the laptop has made connection with the Arduino. This can be checked with the device manager on Windows. On Mac OS, open a terminal window and execute `ls /dev/cu*` on the command line. This will return a list of devices. If only `cu.Bluetooth-Incoming-Port` is present then Mac OS is not connecting to the Arduino. Try connecting and reconnecting the USB data cable. If that fails, try rebooting.

1. Some USB cables are for charging only and not intended for data communications. Verify the cable you are using is a USB data cable.

1. Download and install the Arduino IDE. From the tools menu, select "Arduino Leonardo" for the board, and select the incoming port. Though rare, we have experienced failures of the Arduino devices. Replace the Arduino and try again.

1. The Arduino must have the tclab-sketch firmware to communicate with tclab. Download and install the latest tclab-sketch firmware.

### Heaters don't appear to work

If you observe no change in temperature after the heaters on turned on:

1. Check that your heater power supply is plugged into power and into the top heater shield, not the Arduino.

1. If you're using a power strip, verify the power strip is turned on.

1. The LED indicator should be at 100% if the heaters are on.

1. Check that lab.P1 and/or lab.P2 are set at non-zero values between 0 and 255. Values of about 100 are good for testing.

1. We have experienced occasional failed USB power supplies. Try a different USB power supply.